In [ ]:
%pip install eyepop==3.12.0

### Setting authentication variables

In [ ]:
import getpass

EYEPOP_ACCOUNT_ID=input("Enter your Account UUID: ")
EYEPOP_API_KEY=getpass.getpass('Enter your API KEY: ')

### Setting a namespace prefix

In [ ]:
NAMESPACE_PREFIX="XXXXXXXXXX" # Change this to your namespace prefix

## Defining the abilities
This just defines the list of abilities as prototypes. The registration happens below.

In [ ]:
from eyepop import EyePopSdk
from eyepop.data.data_types import InferRuntimeConfig, VlmAbilityGroupCreate, VlmAbilityCreate, TransformInto
from eyepop.worker.worker_types import CropForward, ForwardComponent, FullForward, InferenceComponent, Pop
import json


ability_prototypes = [
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-classify.detect-nano-watermark",
        description="Classify if image contains an AI watermark",
        worker_release="qwen3-instruct",
        text_prompt="You are given images generated from Nano Banana image generator. Your task is to detect the presence of an AI watermark. Look for a four pointed star or starburst shape, located in the bottom right corner. Assign the label Watermark if any of the following indicators are present, even if they are faint, partially obscured, or blended with the background: 1. There is a white, four-pointed star in the lower right hand side of the image. It is composed of four sharp, symmetrical points, radiating from a central point. 2. The star is rendered in white and looks similar to a starburst or sparkle icon. Assign the label `No_Watermark` in all other cases. Return only one word as the final answer: either `Watermark` or `No_Watermark`.",
        transform_into=TransformInto(
            classes=[
                "Watermark",
                "No_Watermark"
            ]
        ),
        config=InferRuntimeConfig(
            max_new_tokens=5,
            image_size=1000
        ),
        is_public=False
    ),
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-describe.watermark-describe",
        description="Describe the watermarked image",
        worker_release="qwen3-instruct",
        text_prompt="You are given images from generated from Nano Banana image generator. Your task is to describe the symbol at the lower right hand side of the figure. State any defining features. ",
        transform_into=TransformInto(),
        config=InferRuntimeConfig(
            max_new_tokens=100,
            image_size=512
        ),
        is_public=False
    )
]



### Create all abilities, groups and aliases

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID) as endpoint:
    for ability_prototype in ability_prototypes:
        ability_group = endpoint.create_vlm_ability_group(VlmAbilityGroupCreate(
            name=ability_prototype.name,
            description=ability_prototype.description,
            default_alias_name=ability_prototype.name,
        ))
        ability = endpoint.create_vlm_ability(
            create=ability_prototype,
            vlm_ability_group_uuid=ability_group.uuid,
        )
        ability = endpoint.publish_vlm_ability(
            vlm_ability_uuid=ability.uuid,
            alias_name=ability_prototype.name,
        )
        ability = endpoint.add_vlm_ability_alias(
            vlm_ability_uuid=ability.uuid,
            alias_name=ability_prototype.name,
            tag_name="latest"
        )
        print(f"created ability {ability.uuid} with alias entries {ability.alias_entries}")

### Run individual inference via Pop

In [ ]:
example_urls = [
    "https://media.istockphoto.com/id/2187504402/photo/jaguar-f-type-svr-sports-car.jpg?s=612x612&w=0&k=20&c=NdGWdV6tWIwl95k2fw-LxVcn0ZBR73h08-7aYsrChfQ=" # Add more photos to test the ability
]

In [ ]:
pop_classify = Pop(components=[
    InferenceComponent(
        ability=f"{NAMESPACE_PREFIX}.image-classify.detect-nano-watermark:latest"
    )
])
pop_describe = Pop(components=[
    InferenceComponent(
        ability=f"{NAMESPACE_PREFIX}.image-describe.watermark-describe:latest"
    )
])
with EyePopSdk.workerEndpoint(api_key=EYEPOP_API_KEY) as endpoint:
    endpoint.set_pop(pop_classify)
    for url in example_urls:
        job = endpoint.load_from(url)
        while result := job.predict():
            print(json.dumps(result, indent=2))
    print("done")

### Evaluation

In [ ]:
from eyepop.data.data_types import EvaluateRequest

with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID) as endpoint:
    evaluate_req = EvaluateRequest(
        ability_uuid=ability.uuid,
        dataset_uuid="xxxxxxxxxxxxx" # Update with dataset UUID you would like to run the ability against

    )
    evaluation = endpoint.start_vlm_ability_evaluation(
            vlm_ability_uuid=ability.uuid,
            evaluate_request=evaluate_req
    )

    print(evaluation)
    print(evaluation.source)

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID) as endpoint:
    eval = endpoint.get_vlm_ability_evaluation(
      vlm_ability_uuid=ability.uuid,
      source=evaluation.source
    )
    print(eval)